In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import json

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_highest-grossing_films'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
# Locate the main table (first wikitable)
table = soup.find('table', {'class': 'wikitable'})
rows = table.find_all('tr')[1:]  # Skip header

In [4]:
# Database setup
conn = sqlite3.connect('films.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS films
             (id INTEGER PRIMARY KEY AUTOINCREMENT,
              title TEXT NOT NULL,
              release_year INTEGER,
              directors TEXT,
              box_office REAL,
              countries TEXT)''')

# Useful for multiple runs (debugging in my case)
c.execute('DELETE FROM films')

## Film Data Scraper and Database Inserter

This code scrapes film details from Wikipedia pages and inserts the cleaned data into a SQLite database. The process involves the following steps:

1. **Web Scraping**: 
   - For each film, we extract details such as the title, box office revenue, release year, director(s), and country(ies).
   - If a film has a link to its dedicated Wikipedia page, additional details like directors and countries are scraped from that page.

2. **Data Cleaning**:
   - Box office revenue is converted to a numeric value by removing dollar signs (`$`), commas (`,`), and references to other Wikipedia pages (`[1]` for example).
   - Director names and countries are extracted as lists and stored as JSON strings.

3. **Database Insertion**:
   - The cleaned data is inserted into a SQLite database table named `films` with columns: `title`, `release_year`, `directors`, `box_office`, and `countries`.

### Key Functions

- **`scrape_film_details(film_url)`**:
  - Extracts director(s) and country(ies) from the film's Wikipedia page.
  - Handles cases where there are multiple directors or countries listed.
  - Returns director(s) and country(ies) as JSON-encoded strings.

- **Main Loop**:
  - Iterates through rows of a table, extracts relevant data, and calls the scraper function if a film link exists.
  - Inserts the cleaned data into the database.

In [5]:
def scrape_film_details(film_url):
    try:
        response = requests.get(film_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract director(s)
        directors = []
        director_th = soup.find('th', string='Directed by')
        if director_th:
            director_td = director_th.find_next('td')
            # Check if directors are in a plainlist (multiple directors)
            plainlist = director_td.find('div', class_='plainlist')
            if plainlist:
                for li in plainlist.find_all('li'):
                    for sup in li.find_all('sup'):
                        sup.extract()
                    director_name = li.get_text(strip=True)
                    if director_name:
                        directors.append(director_name)
            else:
                # Single director case
                director_name = director_td.get_text(strip=True)
                if director_name:
                    directors.append(director_name)
        
        # Extract country
        countries = []
        country_th = soup.find('th', string='Country') or soup.find('th', string='Countries')
        if country_th:
            country_td = country_th.find_next('td')
            # Check if the label is "Countries" (multiple countries)
            if country_th.get_text(strip=True) == 'Countries':
                # Extract the first country from the list
                country_list = country_td.find('ul')
                if country_list:
                    for country in country_list.find_all('li'):
                        for sup in country.find_all('sup'):
                            sup.extract()
                        if country:
                            countries.append(country.get_text(strip=True))
            else:
                # Single country case
                countries.append(country_td.get_text(strip=True))
        
        return json.dumps(directors), json.dumps(countries)
    except Exception as e:
        print(f"Error scraping {film_url}: {e}")
        return json.dumps([]), json.dumps([])


for row in rows:
    cols = row.find_all(['th', 'td'])
    title_cell = cols[2]
    title = title_cell.get_text(strip=True)

    box_office_cell = cols[3]
    for sup in box_office_cell.find_all('sup'):
        sup.extract()
    box_office = float(box_office_cell.get_text(
        strip=True).replace('$', '').replace(',', ''))
    release_year = int(cols[4].get_text(strip=True))

    film_link = title_cell.find('a')
    directors, country = json.dumps([]), json.dumps([])
    if film_link and film_link.get('href'):
        directors, country = scrape_film_details(
            'https://en.wikipedia.org' + film_link['href'])

    # Insert cleaned data into the database
    c.execute('''INSERT INTO films 
                (title, release_year, directors, box_office, countries)
                VALUES (?, ?, ?, ?, ?)''',
              (title, release_year, directors, box_office, country))

conn.commit()

In [6]:
# Query the database and convert to JSON
c.execute('SELECT * FROM films')
records = c.fetchall()

# Convert rows to a list of dictionaries
films = []
for record in records:
    films.append({
        'id': record[0],
        'title': record[1],
        'release_year': record[2],
        'director': json.loads(record[3]),  # Deserialize JSON string to list
        'box_office': record[4],
        'countries': json.loads(record[5])  # Deserialize JSON string to list
    })
conn.close()

In [7]:
# Export to JSON
with open('./website/data/films.json', 'w') as f:
    json.dump(films, f, indent=4)